In [ ]:
from retrieval import retrieval

In [ ]:
tfidf = retrieval(mode = 'TFIDF') # 리트리버 정의
tfidf.tf.get_sparse_embedding() # 위키 기반의 tfidfvectorizer을 만들고 passge_embedding을 만듭니다.
tfidf.tf.build_faiss() # faiss를 통해 빠른 서치가 가능하도록 faiss index를 만듭니다.
result = tfidf.tf.search_query(mode = 'test') # faiss를 사용하지 않고 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.
result = tfidf.tf.search_query_faiss(mode = 'test') # faiss를 사용하여 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.


dense = retrieval(mode = 'Dense') # 리트리버 정의
dense.dense.load_model() # 모델을 로드합니다.
dense.dense.get_trainer() # 로드하지 않는다면 trainer을 만듭니다.
dense.dense.train() # 모델을 훈련합니다.
dense.dense.train_kfold() # 모델을 K fold 훈련합니다.
dense.dense.build_faiss() # 훈련되거나 로드된 모델의 faiss indexer을 만듭니다.
result = dense.dense.search_query(mode = 'test') # 모델을 통해 결과를 추론합니다. mode = 'test'일 경우 test 데이터셋을, 'eval'일 경우 validation 데이터셋을 통해 정확도를 출력합니다.
result = dense.dense.search_query_faiss(mode = 'test') # 모델을 통해 결과를 추론합니다. mode = 'test'일 경우 test 데이터셋을, 'eval'일 경우 validation 데이터셋을 통해 정확도를 출력합니다.

bm = retrieval(mode = 'BM25')
bm.bm.get_sparse_embedding()
result = bm.bm.search_query(mode = 'test') # 모델을 통해 결과를 추론합니다. mode = 'test'일 경우 test 데이터셋을, 'eval'일 경우 validation 데이터셋을 통해 정확도를 출력합니다.

result = dense.cross_reranking(mode = 'test') # 모델을 통해 결과를 추론합니다. mode = 'test'일 경우 test 데이터셋을, 'eval'일 경우 validation 데이터셋을 통해 정확도를 출력합니다.
result = dense.sequential_reranking(mode = 'test') # 모델을 통해 결과를 추론합니다. mode = 'test'일 경우 test 데이터셋을, 'eval'일 경우 validation 데이터셋을 통해 정확도를 출력합니다.

In [ ]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.train() #model.train()은 기존에 주어진 데이터셋을 활용하여 훈련합니다. model.train(train_dataset, eval_dataset)으로 본인이 직접 지정한 데이터를 훈련할 수도 있습니다.
# 이때, train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
# 모델은 Extraction_based_MRC_outputs + model_name의 폴더로 저장됩니다.
# 모델은 학습이 끝나면 가장 낮은 validaiton loss를 기준으로 저장됩니다.
model.inference(result)
# retrieval을 통해 얻어낸 결과를 predict_reuslt에 json파일 형태로 저장합니다.
"""inference는 반드시
DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
}) 의 형태를 따라야 합니다.
"""
model.load_model() # 체크포인트의 제일 마지막 폴더를 따라가서 trainer_state.json을 열고 거기서 best_model_checkpoint의 경로를 통해 모델을 불러옵니다.
# 불러온 모델은 inference할 수 있습니다.


In [ ]:
import All_dataset
from arguments import Extraction_based_MRC_arguments
datas = All_dataset.prepare_dataset(Extraction_based_MRC_arguments)
# 이처럼 All_dataset.prepare_dataset(argument에서 불러온 모델의 arg)를 통해 각 모델에서 쓰이는 데이터들을 볼 수 있습니다.
# 데이터셋을 할당하지 않으면 않으면 기본으로 주어진 데이터를 바탕으로 데이터를 전처리합니다.
# get_mrc_train_dataset(여러분이 바꾼 데이터) 를 통해 바꾼 데이터를 전처리할 수도 있습니다.
# train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
train = datas.get_mrc_train_dataset()
eval = datas.get_mrc_eval_dataset()

